# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [21]:
# import libraries
import pandas as pd
pd.set_option('display.max_columns', 100)
import os
import re
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.tokenize import sent_tokenize
# Download the punkt tokenizer models (only need to do this once)


from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse_db.db')
df = pd.read_sql_table('DisasterResponse_tbl',con=engine)

# display loaded dataframe
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
# Split dataset
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [24]:
def tokenize_text(text):
    '''
    tonkenize_text
    '''
    
    # Import and load resources inside the function
    import re
    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    from nltk.stem import WordNetLemmatizer

    # Ensure the resources are available in each spawned process
    nltk.download(['stopwords','punkt', 'wordnet','averaged_perceptron_tagger'], quiet=True)
    # Normalize text: lower case and remove punctuations
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatization: Reduce words to their base form
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(word).strip() for word in tokens]

    return clean_tokens   

In [33]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    '''
    class Starting verb extractor
    
    '''

    def starting_verb(self, text):
        # Tokenize sentences
        sentence_list = sent_tokenize(text)

        for sentence in sentence_list:
            # Tokenize words in each sentence
            pos_tags = nltk.pos_tag(word_tokenize(sentence))
            
            # Check if the sentence is not empty
            if not pos_tags:
                continue
            
            # First word and its POS tag
            first_word, first_tag = pos_tags[0]
            
            # Return True if the first word is an appropriate verb type
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Apply starting_verb function to all messages
        X_tagged = pd.Series(X).apply(self.starting_verb)

        return pd.DataFrame(X_tagged)

In [26]:
# class WordCountExtractor(BaseEstimator, TransformerMixin):
#     def fit(self, x, y=None):
#         return self

#     def transform(self, texts):
#         return [[len(text.split())] for text in texts]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [27]:
# def ML_pipeline(clf = AdaBoostClassifier()):
#     pipeline = Pipeline([
#         ('features', FeatureUnion([
#             ('text_pipeline', Pipeline([
#                 ('vect', CountVectorizer(tokenizer=tokenize)),
#                 ('tfidf', TfidfTransformer())
#             ]))
#         ])),
#         ('clf', MultiOutputClassifier(clf))
#     ])
    
#     return pipeline

# # include StartingVerbExtractor custom transformer
# def ML_pipeline_2(clf = AdaBoostClassifier()):
#     pipeline = Pipeline([
#         ('features', FeatureUnion([
#             ('text_pipeline', Pipeline([
#                 ('vect', CountVectorizer(tokenizer=tokenize)),
#                 ('tfidf', TfidfTransformer())
#             ])),
#             ('starting_verb', StartingVerbExtractor())
#         ])),
#         ('clf', MultiOutputClassifier(clf))
#     ])
    
#     return pipeline

# Define the machine learning pipeline
def build_model():
    '''
    Random Forest Pipeline
    '''
    ml_rf_pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize_text)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    return ml_rf_pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = build_model()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [29]:
y_pred_test = model.predict(X_test)

# classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y_test.columns.values))
print("------------------------------------------------------\n")

                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      4055
               request       0.79      0.44      0.57       923
                 offer       0.00      0.00      0.00        27
           aid_related       0.74      0.58      0.65      2207
          medical_help       0.63      0.09      0.15       426
      medical_products       0.73      0.07      0.12       238
     search_and_rescue       0.59      0.10      0.17       161
              security       0.17      0.01      0.02        88
              military       0.44      0.04      0.08       166
                 water       0.89      0.28      0.43       387
                  food       0.83      0.53      0.65       611
               shelter       0.82      0.30      0.44       476
              clothing       0.83      0.07      0.13        74
                 money       0.71      0.04      0.07       128
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [30]:
# get list of model parameters
model.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline__vect__token_pattern', 'features__text_pipeline__vect__tokenizer', 

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define parameters to tune
parameters = {
    'clf__estimator__n_estimators': [20, 30],
    'clf__estimator__max_depth': [None, 10],  
    'clf__estimator__min_samples_split': [2, 4]  
}


cv = GridSearchCV(model, param_grid=parameters, verbose=3, n_jobs=-1, cv=3)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=30 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=30 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=30 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20 
[CV] clf__estimator__max_depth=None, clf__estimator__min_samples_split=4, clf__estim

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_test = cv.predict(X_test)

# classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [34]:
# try other model -> Adaboost + StartingVerbExtractor

def ada_pipeline(clf = AdaBoostClassifier()):
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize_text)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartingVerbExtractor())
        ])),
        ('clf', MultiOutputClassifier(clf))
    ])
    
    return pipeline

ada_model = ada_pipeline()
ada_model.fit(X_train, y_train)
y_pred_ada_test = ada_model.predict(X_test)

print(classification_report(y_test.values, y_pred_ada_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.80      0.98      0.88      4055
               request       0.77      0.51      0.61       923
                 offer       0.00      0.00      0.00        27
           aid_related       0.77      0.60      0.67      2207
          medical_help       0.60      0.25      0.35       426
      medical_products       0.65      0.33      0.44       238
     search_and_rescue       0.73      0.22      0.33       161
              security       0.20      0.03      0.06        88
              military       0.51      0.26      0.34       166
                 water       0.79      0.60      0.68       387
                  food       0.80      0.63      0.70       611
               shelter       0.79      0.53      0.64       476
              clothing       0.72      0.31      0.43        74
                 money       0.52      0.21      0.30       128
        missing_people       0.58      

### 9. Export your model as a pickle file

In [35]:
# save model in pickle file
with open('classifier.pkl', 'wb') as f:
    pickle.dump(ada_model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.